# Twitch API Request Test
---
Twitch API Docs : [Twitch Developers] (https://dev.twitch.tv/docs/v5) <br>
Twitch API Client : [Twitch Developers Console] (https://dev.twitch.tv/console/apps) <br>
Twitch API Video Obj : [Twitch API Video Reference] (https://dev.twitch.tv/docs/api/reference#get-videos) <br>
Test Live Video : [PAKA 11/03 Broadcast](https://www.twitch.tv/videos/790636820) 

In [1]:
import requests
import json
import sys

In [2]:
videoID = "790636820"
clientID = "k8tbrzzuqt746l6zq8wqdbcad06tb0"

params = dict({"client_id":clientID})

In [3]:
fileHandler = open("Paka1103.txt", "wt", encoding="utf-8")

In [4]:
time = list()
user = list()
chat = list()

i = 0
while True:
    if i == 0:
        URL = 'https://api.twitch.tv/v5/videos/' + videoID + '/comments?content_offset_seconds=0'
        i += 1
    else:
        URL = 'https://api.twitch.tv/v5/videos/' + videoID + '/comments?cursor='
        URL += nextCursor

    response = requests.get(URL, params=params)

    data = json.loads(response.text)
    
    # append in list
    for idx in range(len(data["comments"])):
        Time = data["comments"][idx]["created_at"][11:19]
        User = data["comments"][idx]["commenter"]["display_name"]
        Chat = data["comments"][idx]["message"]["body"]
        
        #print(Time, User, Chat)
        time.append(Time)
        user.append(User)
        chat.append(Chat)
    
    if "_next" not in data.keys():
        break
        
    nextCursor = data["_next"]